<a href="https://colab.research.google.com/github/yishingene/AIA-reading/blob/master/sider_220912_%223_hops%22_balance_10_fold_MLP_2_layer_cross_validate_negative_example_CrossEntropy_link_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# use negative example (not negative sampling!!s)
# ref https://docs.dgl.ai/en/latest/new-tutorial/4_link_predict.html

In [2]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
# import itertools
# import scipy.sparse as sp
import dgl
import dgl.nn as dglnn
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import numpy as np
from dgl.data.utils import generate_mask_tensor
from dgl.data import DGLDataset
import torch
import scipy.sparse as sp
# from sklearn.metrics import roc_auc_score,precision_recall_curve, auc, ndcg_score, mean_squared_error
from sklearn.metrics import roc_auc_score,precision_recall_curve, auc, f1_score,precision_score, recall_score,average_precision_score,label_ranking_average_precision_score
from sklearn import metrics
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("/content/drive/MyDrive/sideeffects/data/side-effect-and-drug_name.tsv",sep = "\t")
drug_id = df["drugbank_id"]
drug_name = df["drugbank_name"] 
side_effect =df["side_effect_name"] 
edgelist1 = zip(side_effect, drug_name)
dfs = df[['drugbank_name','side_effect_name']]

col_names = ["left_side","right_side","similairity"]
drugsim = pd.read_csv("/content/drive/MyDrive/sideeffects/data/semantic_similarity_side_effects_drugs.txt",sep ="\t",
                 names =col_names, header=None)
source =drugsim["left_side"]
destination = drugsim["right_side"]
similarity = drugsim["similairity"]
###Drugs similarity Network#####
edge_list = zip(source,destination,similarity)

from sklearn.preprocessing import LabelEncoder , LabelBinarizer
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# lb = preprocessing.LabelBinarizer()
# dfs : drug-sideeffect, drug_id: one_hot_encode
le.fit(dfs['drugbank_name'])
dfs['drug_id']=le.transform(dfs['drugbank_name']) 
drugsim['drug_id_left']=le.transform(drugsim['left_side'])
drugsim['drug_id_right']=le.transform(drugsim['right_side'])
# lb.fit(dfs['drug_id'])
# dfs['drug_id_onehot']=pd.DataFrame([[i] for i in lb.transform(dfs['drug_id'])])
# drugsim['drugid_left_onehot']=pd.DataFrame([[i] for i in lb.transform(drugsim['drug_id_left'])])
# drugsim['drugid_right_onehot']=pd.DataFrame([[i] for i in lb.transform(drugsim['drug_id_right'])])
# drugsim['left_side']
le.fit(dfs['side_effect_name'])
dfs['se_id']=le.transform(dfs['side_effect_name'])
# dfs['se_id_onehot']=pd.DataFrame([[i] for i in lb.transform(dfs['se_id'])])
# drugsim: drug-drug similarity

 # drug-side effect: dfs
drug_id = torch.LongTensor(dfs['drug_id'])
side_id = torch.LongTensor(dfs['se_id'])

# drug-drug similar: drugsim
src = torch.LongTensor(drugsim['drug_id_left']) 
dst = torch.LongTensor(drugsim['drug_id_right'])   

 # Build graph
G = dgl.heterograph({
    # Heterogeneous graphs are organized as a dictionary of edges connecting two types of nodes.
    # We specify the edges of a type simply with a pair of user ID array and item ID array.
    ('drug_id', 'relate', 'side_id'): (drug_id, side_id),
    # Since DGL graphs are directional, we need an inverse relation from items to users as well.
    ('side_id', 'relate-by', 'drug_id'): (side_id, drug_id),
    ('drug_id','similar','drug_id'):(src,dst),
    ('drug_id','similar-by','drug_id'):(dst,src)
})

edge_weight2 = torch.tensor(drugsim['similairity'].to_numpy()) # drug-drug similar
G.edges['similar'].data['sim'] = edge_weight2
G.edges['similar-by'].data['sim'] = edge_weight2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
G

Graph(num_nodes={'drug_id': 1020, 'side_id': 5599},
      num_edges={('drug_id', 'relate', 'side_id'): 133750, ('drug_id', 'similar', 'drug_id'): 519690, ('drug_id', 'similar-by', 'drug_id'): 519690, ('side_id', 'relate-by', 'drug_id'): 133750},
      metagraph=[('drug_id', 'side_id', 'relate'), ('drug_id', 'drug_id', 'similar'), ('drug_id', 'drug_id', 'similar-by'), ('side_id', 'drug_id', 'relate-by')])

In [6]:
# 從 node pair 找 edge id
G.edge_ids(torch.tensor([0, 0]), torch.tensor([3410, 1008]),etype='relate')

tensor([100,  30])

In [7]:
G.find_edges(torch.tensor([100, 30,8]), 'relate-by')

(tensor([3410, 1008,  236]), tensor([0, 0, 0]))

In [8]:
# 從 edge ID return node pair, 找 Node Pair
G.find_edges(torch.tensor([100, 30,8]), 'relate')

(tensor([0, 0, 0]), tensor([3410, 1008,  236]))

In [9]:
u,v=G.edges(etype='relate')

In [10]:
eids = np.arange(G.number_of_edges(('drug_id', 'relate', 'side_id'))) # put id on edges ('drug_id', 'relate', 'side_id')
eids

array([     0,      1,      2, ..., 133747, 133748, 133749])

In [11]:
# Find all negative edges and split them for training and testing
# 使用 u, v 當作 coordinate, 填入 '1'
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy()))) # 把 source node, dst node 放進 sparse matrix, 
# 有連接的為1, 沒有連接的為0

In [12]:
# adj.toarray()

In [13]:
adj_neg = 1 - adj.todense() # 未連接為1 , 已連接為 0
adj_neg

matrix([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]])

In [14]:
neg_u, neg_v = np.where(adj_neg != 0) # 找出未連接 node

In [15]:
len(v)/(len(v)+len(neg_u)) # link 在 graph 中佔比

0.023419798353347412

In [16]:
# eids_pm = np.random.permutation(eids) # 亂數排列

In [17]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h contains the node representations for each node type computed from
        # the GNN defined in the previous section (Section 5.1).
        with graph.local_scope():
            # print('h:',h)
            # print('graph',graph)
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']


class HeteroMLPPredictor(nn.Module):
    def __init__(self, in_dims, n_classes):
        super().__init__()
        self.W = nn.Linear(in_dims * 2, n_classes)

    def apply_edges(self, edges):
        x = torch.cat([edges.src['h'], edges.dst['h']], 1)
        y = self.W(x)
        return {'score': y}

    def forward(self, graph, h,etype):
        with graph.local_scope():
            graph.ndata['h'] = h   
            graph.apply_edges(self.apply_edges, etype=etype)
            return graph.edges[etype].data['score']


# 2 layer
class Hetero2MLPPredictor(nn.Module):
    def __init__(self, in_dims, n_classes):
        super().__init__()
        self.W1 = nn.Linear(in_dims * 2, in_dims)
        self.W2 = nn.Linear(in_dims, n_classes)
    def apply_edges(self, edges):
        x = torch.cat([edges.src['h'], edges.dst['h']], 1)
        score = self.W2(F.relu(self.W1(x)))
        return {'score': score}

    def forward(self, graph, h,etype):
        with graph.local_scope():
            graph.ndata['h'] = h   
            graph.apply_edges(self.apply_edges, etype=etype)
            return graph.edges[etype].data['score']

In [18]:

import dgl.nn as dglnn
# https://zhuanlan.zhihu.com/p/126149099
# https://docs.dgl.ai/guide/training-node.html?highlight=rgcn

class RGCN(nn.Module): 
    def __init__(self, in_feats, hid_feats, out_feats, rel_names): # in=10,hid=20,out=5,
        super().__init__()
        # featureless embedding
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_feats)) # in=10
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layer
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats) # 10,20
            for rel in rel_names}, aggregate='sum')
        
        self.conv2 = dglnn.HeteroGraphConv({
          rel: dglnn.GraphConv(hid_feats, hid_feats) # 20,20
          for rel in rel_names}, aggregate='sum')
                
        self.conv3 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats) # 20,5
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph):
        # inputs are features of nodes
        h = self.conv1(graph, self.embed)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv3(graph, h)
        return h

In [19]:
fprd = dict()
tprd = dict()
roc_aucd = dict()
precd = dict()
recalld = dict()
auc_prd = dict()

def compute_auc(pos_score, neg_score):
    # print('positive score:',pos_score[:5])
    # print('negative score:',neg_score[:5])
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    # print('scores:',scores)
    # print('labels:',labels)
    # ndcg = ndcg_score([labels], [scores.squeeze(1)], k=5)
    # rmse = mean_squared_error(labels,scores, squared=False)
    prediction_int = np.zeros_like(scores)
    prediction_int[scores > 0.5] = 1
    f1_micro = f1_score(labels, prediction_int, average='micro',zero_division=0)
    f1_macro = f1_score(labels, prediction_int, average='macro',zero_division=0)
    f1_weight = f1_score(labels, prediction_int, average='weighted',zero_division=0)
    prec_micro = precision_score(labels, prediction_int, average='micro',zero_division=0)
    prec_macro = precision_score(labels, prediction_int, average='macro',zero_division=0)
    prec_weight = precision_score(labels, prediction_int, average='weighted',zero_division=0)
    recall_micro = recall_score(labels, prediction_int, average='micro',zero_division=0)
    recall_macro = recall_score(labels, prediction_int, average='macro',zero_division=0)
    recall_weight = recall_score(labels, prediction_int, average='weighted',zero_division=0)

    prec, recall, _ = precision_recall_curve(labels,scores)
    pr_auc_score = auc(recall, prec)

    fpr, tpr, _ = metrics.roc_curve(labels,scores)
    roc_auc_score = metrics.auc(fpr, tpr)

    # fprd[f] = fpr # algorithm
    # tprd[f] = tpr # algorithm
    # roc_aucd[f] = roc_auc_score # algorithm
    # precd[f] = prec # algorithm
    # recalld[f] = recall # algorithm
    # auc_prd[f] = pr_auc_score # algorithm

    return roc_auc_score, pr_auc_score, f1_micro, f1_macro, f1_weight, prec_micro, prec_macro, prec_weight,\
     recall_micro,recall_macro, recall_weight,prec, recall, fpr, tpr




In [20]:

class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        # self.pred = HeteroDotProductPredictor()
        # self.pred = MLPPredictor(hidden_features)
        self.pred = Hetero2MLPPredictor(out_features, 1)
    def forward(self, g, pos_g,neg_g, etype):
        h = self.sage(g) # process incomplete G, output information
        pred_pos_g = self.pred(pos_g, h, etype) # use information inferring pos_g graph
        pred_neg_g = self.pred(neg_g, h, etype) # use information inferreing neg_g graph
        # print("pred_pos_g",torch.sigmoid(pred_pos_g)) # use sigmoid transform to get prediction label
        # print("pred_neg_g",torch.sigmoid(pred_neg_g))
        # with torch.no_grad():
        #     valid_pos_score = self.pred(valid_pos_g, h, etype) # 這裡的 h 是全局的 G 所輸出的 h
        #     valid_neg_score = self.pred(valid_neg_g, h, etype)
            # roc_auc_score,pr_auc_score = compute_auc(pos_score, neg_score)
            # print('AUC-ROC:', roc_auc_score)
            # print('AUC-PR:', pr_auc_score)
            
        return torch.sigmoid(pred_pos_g), torch.sigmoid(pred_neg_g)

In [21]:
# https://docs.dgl.ai/en/latest/new-tutorial/4_link_predict.html
# use cross entropy

# pred = HeteroDotProductPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).squeeze(1)
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    # print('pos_score length:',pos_score.shape[0],'neg_score length:',neg_score.shape[0])
    return F.binary_cross_entropy_with_logits(scores, labels)

In [22]:
# def setdiff2d(A,B): # set difference of two 2d array
#     return np.array(list(set(map(tuple, A.T)) - set(map(tuple, B.T)))).T

In [23]:
len(neg_u)

5577230

In [24]:
# neg_eids = np.random.choice(len(neg_u), G.number_of_edges(('drug_id', 'relate', 'side_id')) // 2)
neg_eids = np.random.choice(len(neg_u), G.number_of_edges(('drug_id', 'relate', 'side_id'))) # 找與 positive 同樣長度的 , 這裡之後可以用 sampling, 重複測試
# neg_eids = np.random.choice(len(neg_u), G.number_of_edges(('drug_id', 'relate', 'side_id'))*50 ) # 1:50
neg_eids.shape

(133750,)

In [25]:
FOLDS = 10
sz = G.number_of_edges(('drug_id', 'relate', 'side_id'))
eids = np.arange(sz) # list: all positive edges ids, 133750個
eids_pm = eids
test_size = fsz = int(sz /10) # 133750 => 13775 , test size, 可以從這裡開始改 train, cross validate 10 fold  , test size 10%
np.random.shuffle(eids_pm)
neg_eids_pm = neg_eids
# np.random.shuffle(neg_eids_pm)
IDX = u,v
IDX_neg = neg_u,neg_v
offset = 0
AUC_roc_train = np.zeros(FOLDS)
AUC_roc_valid = np.zeros(FOLDS)
AUC_roc_test = np.zeros(FOLDS)
AUC_pr_train = np.zeros(FOLDS)
AUC_pr_valid = np.zeros(FOLDS)
AUC_pr_test = np.zeros(FOLDS)
F1_micro_test = np.zeros(FOLDS)
F1_macro_test = np.zeros(FOLDS)
F1_weight_test = np.zeros(FOLDS)
Prec_micro_test = np.zeros(FOLDS)
Prec_macro_test = np.zeros(FOLDS)
Prec_weight_test  = np.zeros(FOLDS)
Recall_micro_test = np.zeros(FOLDS)
Recall_macro_test = np.zeros(FOLDS)
Recall_weight_test = np.zeros(FOLDS) 

In [26]:
algod = {}

In [27]:
for f in range(FOLDS):
    print("== Fold:",f," ==")
    # 取 test set,佔1/5
    test_pos_u, test_pos_v = idx_test = u[eids_pm[offset:offset+fsz]], v[eids_pm[offset:offset+fsz]]
    # print('length of test_pos',len(test_pos_u))
    # test_neg_u, test_neg_v = idx_test_neg = neg_u[neg_eids_pm[offset:offset+fsz*50]], neg_v[neg_eids_pm[offset:offset+fsz*50]] # 1:50
    test_neg_u, test_neg_v = idx_test_neg = neg_u[neg_eids_pm[offset:offset+fsz]], neg_v[neg_eids_pm[offset:offset+fsz]]
    # print('length of test_neg',len(test_neg_u))
    # validation set
    # validate_pos_u, validate_pos_v = u[eids_pm[offset + fsz:offset + 2*fsz]], v[eids_pm[offset + fsz:offset + 2*fsz]]
    # print('length of validate_pos ',len(validate_pos_u))
    # validate_neg_u, validate_neg_v = neg_u[neg_eids_pm[offset + fsz:offset + 2*fsz]], neg_v[neg_eids_pm[offset + fsz:offset + 2*fsz]]
    # print('length of validate_neg ',len(validate_neg_u))

    # training set
    train_pos_u, train_pos_v = u[eids_pm[np.r_[:offset,offset + fsz:len(eids_pm)]]], v[eids_pm[np.r_[:offset,offset + fsz:len(eids_pm)]]]
    # print('length of train_pos ',len(train_pos_u))
    train_neg_u, train_neg_v = neg_u[neg_eids_pm[np.r_[:offset,offset + fsz:len(neg_eids_pm)]]], neg_v[neg_eids_pm[np.r_[:offset,offset + fsz:len(neg_eids_pm)]]]
    # print('length of train_neg ',len(train_neg_u))

    # sub Graph: train_pos_g
    num_nodes_dict = {'drug_id': 1020, 'side_id': 5599}
    train_pos_g = dgl.heterograph({
        ('drug_id', 'relate', 'side_id'): (train_pos_u, train_pos_v),
        ('side_id', 'relate-by', 'drug_id'): (train_pos_v, train_pos_u),
        ('drug_id','similar','drug_id'):(src,dst),
        ('drug_id','similar-by','drug_id'):(dst,src)
        
    },num_nodes_dict=num_nodes_dict)
    train_pos_g.edges['similar'].data['sim'] = edge_weight2
    train_pos_g.edges['similar-by'].data['sim'] = edge_weight2

    # sub Graph: train_neg_g 
    train_neg_g = dgl.heterograph({
    ('drug_id', 'relate', 'side_id'): (train_neg_u, train_neg_v),
    ('side_id', 'relate-by', 'drug_id'): (train_neg_v, train_neg_u),
    ('drug_id','similar','drug_id'):(src,dst),
    ('drug_id','similar-by','drug_id'):(dst,src)

    },num_nodes_dict=num_nodes_dict)
    train_neg_g.edges['similar'].data['sim'] = edge_weight2
    train_neg_g.edges['similar-by'].data['sim'] = edge_weight2

    # valid_pos_g = dgl.heterograph({
    #     ('drug_id', 'relate', 'side_id'): (validate_pos_u, validate_pos_v),
    #     ('side_id', 'relate-by', 'drug_id'): (validate_pos_v, validate_pos_u),
    #     ('drug_id','similar','drug_id'):(src,dst),
    #     ('drug_id','similar-by','drug_id'):(dst,src)
        
    # },num_nodes_dict=num_nodes_dict)
    # valid_pos_g.edges['similar'].data['sim'] = edge_weight2
    # valid_pos_g.edges['similar-by'].data['sim'] = edge_weight2


    # valid_neg_g = dgl.heterograph({
    #     ('drug_id', 'relate', 'side_id'): (validate_neg_u, validate_neg_v),
    #     ('side_id', 'relate-by', 'drug_id'): (validate_neg_v, validate_neg_u),
    #     ('drug_id','similar','drug_id'):(src,dst),
    #     ('drug_id','similar-by','drug_id'):(dst,src)
        
    # },num_nodes_dict=num_nodes_dict)
    # valid_neg_g.edges['similar'].data['sim'] = edge_weight2
    # valid_neg_g.edges['similar-by'].data['sim'] = edge_weight2

    # sub Graph: test_pos_g
    test_pos_g = dgl.heterograph({
    ('drug_id', 'relate', 'side_id'): (test_pos_u, test_pos_v),
    ('side_id', 'relate-by', 'drug_id'): (test_pos_v, test_pos_u),
    ('drug_id','similar','drug_id'):(src,dst),
    ('drug_id','similar-by','drug_id'):(dst,src)

    },num_nodes_dict=num_nodes_dict)
    test_pos_g.edges['similar'].data['sim'] = edge_weight2
    test_pos_g.edges['similar-by'].data['sim'] = edge_weight2

    # sub Graph: test_neg_g
    test_neg_g = dgl.heterograph({
    ('drug_id', 'relate', 'side_id'): (test_neg_u, test_neg_v),
    ('side_id', 'relate-by', 'drug_id'): (test_neg_v, test_neg_u),
    ('drug_id','similar','drug_id'):(src,dst),
    ('drug_id','similar-by','drug_id'):(dst,src)

    },num_nodes_dict=num_nodes_dict)
    test_neg_g.edges['similar'].data['sim'] = edge_weight2
    test_neg_g.edges['similar-by'].data['sim'] = edge_weight2 

    # train_g: remove test set only, offset:offset+fsz
    train_g = dgl.remove_edges(G, eids_pm[offset:offset+fsz],'relate') 

    train_g = dgl.remove_edges(train_g, eids_pm[offset:offset+fsz],'relate-by') #因為對稱, 所以 edge ID 一樣,'relate'移除掉的 edge ID, 不會影響到 'relate-by'的 ID

    model = Model(10, 20, 5, train_g.etypes) # inputfeatures=10, hidden_features=20, output_features=1, relation_names: train_g.etypes
    opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    # i=0
    for epoch in range(300):
        pos_score, neg_score = model(train_g,train_pos_g, train_neg_g, ('drug_id', 'relate', 'side_id'))
        loss = compute_loss(pos_score, neg_score)
        opt.zero_grad()
        loss.backward()
        opt.step()
        # i=i+1
        if epoch % 20==0:
            print('the',epoch,'loss=',loss.item())
            with torch.no_grad(): # 這裡似乎弄錯了, 要使用 test set, 然後在 model 中使用 validation set , 
                # pos_score = self.pred(test_pos_g, h, etype) # 這裡的 h 是全局的 G 所輸出的 h
                # neg_score = self.pred(test_neg_g, h, etype)
                # roc_auc_score_train,pr_auc_score_train, ndcg, rmse = compute_auc(pos_score, neg_score,f)
                # roc_auc_score_valid,pr_auc_score_valid = compute_auc(valid_pos_score, valid_neg_score)
                # print('AUC-ROC-TRAIN:', roc_auc_score_train,'AUC-PR-TRAIN:', pr_auc_score_train,'ndcg-TRAIN:',ndcg,'rmse-TRAIN:',rmse)
                # print('AUC-ROC-VALID:', roc_auc_score_valid,'AUC-PR-VALID:', pr_auc_score_valid)
                roc_auc_score_train, pr_auc_score_train , f1_micro_train, _ ,  _ ,  _ ,  _ ,  _ ,  _ , _ ,  _,\
                prec, recall, fpr, tpr= compute_auc(pos_score, neg_score)
                print('roc_auc:',roc_auc_score_train,'f1 micro:',f1_micro_train)
        # print('==',i,' epoch:',loss.item())

    # test set
    with torch.no_grad():
        print("Result:")
        print('AUC-ROC-TRAIN:', roc_auc_score_train,'AUC-PR-TRAIN:', pr_auc_score_train)
        AUC_roc_train[f] = roc_auc_score_train
        AUC_pr_train[f] = pr_auc_score_train
        # print('AUC-ROC-VALID:', roc_auc_score_valid,'AUC-PR-VALID:', pr_auc_score_valid)
        # AUC_roc_valid[f] = roc_auc_score_valid
        # AUC_pr_valid[f] = pr_auc_score_valid
        pos_score, neg_score = model(train_g,test_pos_g, test_neg_g, ('drug_id', 'relate', 'side_id'))
        # roc_auc_score_test,pr_auc_score_test, ndcg_test, rmse_test = compute_auc(pos_score, neg_score,f)
        # print('AUC-ROC-TEST:', roc_auc_score_test,'AUC-PR-TEST:', pr_auc_score_test,'ndcg TEST:',ndcg_test,'rmse TEST:',rmse_test)
        # AUC_roc_test[f] = roc_auc_score_test
        # AUC_pr_test[f] = pr_auc_score_test
        roc_auc_score_test,pr_auc_score_test, f1_micro_test, f1_macro_test, f1_weight_test, prec_micro_test,\
        prec_macro_test, prec_weight_test, recall_micro_test,recall_macro_test, recall_weight_test\
        ,prec, recall, fpr, tpr = compute_auc(pos_score, neg_score)
        print('AUC-ROC-TEST:', roc_auc_score_test,'AUC-PR-TEST:', pr_auc_score_test)
        AUC_roc_test[f] = roc_auc_score_test
        AUC_pr_test[f] = pr_auc_score_test
        F1_micro_test[f] = f1_micro_test
        F1_macro_test[f]  = f1_macro_test
        F1_weight_test[f]  = f1_weight_test
        Prec_micro_test[f]  = prec_micro_test
        Prec_macro_test[f]  = prec_macro_test
        Prec_weight_test[f]   = prec_weight_test
        Recall_micro_test[f]  = recall_micro_test
        Recall_macro_test[f]  = recall_macro_test
        Recall_weight_test[f] = recall_weight_test

        algod['fold'+str(f)] = {}
        algod['fold'+str(f)]['gcnmlp'] = {}
        algod['fold'+str(f)]['gcnmlp']['fpr'] = fpr
        algod['fold'+str(f)]['gcnmlp']['tpr'] = tpr
        algod['fold'+str(f)]['gcnmlp']['roc_auc'] = roc_auc_score_test
        algod['fold'+str(f)]['gcnmlp']['prec'] = prec
        algod['fold'+str(f)]['gcnmlp']['recall'] = recall
        algod['fold'+str(f)]['gcnmlp']['auc_pr'] = pr_auc_score_test
        algod['fold'+str(f)]['gcnmlp']['F1_macro'] = f1_macro_test
        algod['fold'+str(f)]['gcnmlp']['Prec_macro'] = prec_macro_test
        algod['fold'+str(f)]['gcnmlp']['Recall_macro'] = recall_macro_test
    offset += fsz

# print("Mean AUC ROC TRAIN", AUC_roc_train.mean()," ", "Standard Deviation:", AUC_roc_train.std())
# # print("Mean AUC ROC VALID", AUC_roc_valid.mean()," ", "Standard Deviation:", AUC_roc_valid.std())
# print("Mean AUC ROC TEST", AUC_roc_test.mean()," ", "Standard Deviation:", AUC_roc_test.std())
# print("Mean AUC PR TRAIN", AUC_pr_train.mean()," ", "Standard Deviation:", AUC_pr_train.std())
# # print("Mean AUC PR VALID", AUC_pr_valid.mean()," ", "Standard Deviation:", AUC_pr_valid.std())
# print('Mean AUC PR TEST', AUC_pr_test.mean()," ", "Standard Deviation:", AUC_pr_test.std())
print("Mean AUC ROC TEST", AUC_roc_test.mean()," ", "SD:", AUC_roc_train.std())
print("Mean AUC PR TEST", AUC_pr_test.mean()," ", "SD:", AUC_pr_train.std())
print("f1 micro test", F1_micro_test.mean()," ", "SD:", F1_micro_test.std())
print("f1_macro_test", F1_macro_test.mean()," ", "SD:", F1_macro_test.std())
print("f1_weight_test", F1_weight_test.mean()," ", "SD:", F1_weight_test.std())
print("prec_micro_test", Prec_micro_test.mean()," ", "SD:", Prec_micro_test.std())
print("prec_macro_test", Prec_macro_test.mean()," ", "SD:", Prec_macro_test.std())
print("prec_weight_test", Prec_weight_test.mean()," ", "SD:", Prec_weight_test.std())
print("recall_micro_test", Recall_micro_test.mean()," ", "SD:", Recall_micro_test.std())
print("recall_macro_test", Recall_macro_test.mean()," ", "SD:", Recall_macro_test.std())
print("recall_weight_test", Recall_weight_test.mean()," ", "SD:", Recall_weight_test.std())

== Fold: 0  ==


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


the 0 loss= 0.7280669212341309
roc_auc: 0.5806965627705908 f1 micro: 0.5


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


the 20 loss= 0.6108787059783936
roc_auc: 0.9276869697736285 f1 micro: 0.8096863966770509


...

the 260 loss= 0.5642661452293396
roc_auc: 0.9413469578732954 f1 micro: 0.8662845275181724


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


the 280 loss= 0.5642525553703308
roc_auc: 0.9413121709783269 f1 micro: 0.8652502596053998


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Result:
AUC-ROC-TRAIN: 0.9413121709783269 AUC-PR-TRAIN: 0.9395876630628878
AUC-ROC-TEST: 0.936404786444231 AUC-PR-TEST: 0.935803410913221
Mean AUC ROC TEST 0.9331092833959298   SD: 0.010910957172725074
Mean AUC PR TEST 0.9377835373975216   SD: 0.004868678321714095
f1 micro test 0.8640710280373831   SD: 0.0033307739905336195
f1_macro_test 0.863713400109668   SD: 0.0033670080993411272
f1_weight_test 0.863713400109668   SD: 0.003367008099341093
prec_micro_test 0.8640710280373831   SD: 0.003330773990533644
prec_macro_test 0.8679234050197022   SD: 0.003094816210452661
prec_weight_test 0.8679234050197022   SD: 0.003094816210452662
recall_micro_test 0.8640710280373831   SD: 0.003330773990533644
recall_macro_test 0.8640710280373831   SD: 0.003330773990533644
recall_weight_test 0.8640710280373831   SD: 0.003330773990533644


In [28]:
model.eval()

Model(
  (sage): RGCN(
    (embed): ParameterDict(
        (drug_id): Parameter containing: [torch.FloatTensor of size 1020x10]
        (side_id): Parameter containing: [torch.FloatTensor of size 5599x10]
    )
    (conv1): HeteroGraphConv(
      (mods): ModuleDict(
        (relate): GraphConv(in=10, out=20, normalization=both, activation=None)
        (similar): GraphConv(in=10, out=20, normalization=both, activation=None)
        (similar-by): GraphConv(in=10, out=20, normalization=both, activation=None)
        (relate-by): GraphConv(in=10, out=20, normalization=both, activation=None)
      )
    )
    (conv2): HeteroGraphConv(
      (mods): ModuleDict(
        (relate): GraphConv(in=20, out=20, normalization=both, activation=None)
        (similar): GraphConv(in=20, out=20, normalization=both, activation=None)
        (similar-by): GraphConv(in=20, out=20, normalization=both, activation=None)
        (relate-by): GraphConv(in=20, out=20, normalization=both, activation=None)
      )

In [29]:
# from torchsummary import summary

In [30]:
# summary(model,(1,10))

In [31]:
algod['fold0']

{'gcnmlp': {'fpr': array([0.00000000e+00, 2.24299065e-04, 3.73831776e-04, ...,
         9.99028037e-01, 9.99177570e-01, 1.00000000e+00]),
  'tpr': array([0.        , 0.09517757, 0.13106542, ..., 1.        , 1.        ,
         1.        ]),
  'roc_auc': 0.9394298611232422,
  'prec': array([0.50063632, 0.50061763, 0.50063637, ..., 0.99715586, 0.9976489 ,
         1.        ]),
  'recall': array([1.        , 0.99992523, 0.99992523, ..., 0.13106542, 0.09517757,
         0.        ]),
  'auc_pr': 0.941858828547657,
  'F1_macro': 0.8643230915401123,
  'Prec_macro': 0.8689215289391291,
  'Recall_macro': 0.8647102803738318}}

In [32]:
import os
import sys
import pickle
# file =  "/content/drive/MyDrive/sideeffects/data/algod_gcnmlp2layer3hop0731.pickle"
file =  "/content/drive/MyDrive/sideeffects/data/algod_gcnmlp2layer3hop0731.pickle"
if os.path.exists(file): # check file exist
    print('檔案已經存在')
    print(file)
    sys.exit()
with open(file, 'wb') as f:
    pickle.dump(algod, f)

檔案已經存在
/content/drive/MyDrive/sideeffects/data/algod_gcnmlp2layer3hop0731.pickle


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Load
with open(file, 'rb') as f:
    new_dict = pickle.load(f)

In [ ]:
!pip install twitter

In [ ]:
from twitter import *

OAUTH_TOKEN = '797713376314916864-psiiaKesuk4osHa0ohx05021YUeTWgx'
OAUTH_SECRET = 'BAHQUgsvlwHgFFmQYoSFj91mWImcnc0l7wIcODcX7IrR6'
CONSUMER_KEY = 'UBdUJkJBvKN5SueswwgAVBa2N'
CONSUMER_SECRET = 'zY1brqqXvhRgzPw6Gunx7lEU7AYwNECQIIf8polq2rBtGOQpbj'

t = Twitter(auth=OAuth(OAUTH_TOKEN, OAUTH_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

# Send a direct message
def send_twitter(msg):
    t.direct_messages.events.new(
        _json={
            "event": {
                "type": "message_create",
                "message_create": {
                    "target": {
                        "recipient_id": t.users.show(screen_name="ystwer")["id"]},
                    "message_data": {
                        "text": msg}}}})

send_twitter("程式煮好了")
# 送訊息到 sasmpr